<a href="https://colab.research.google.com/github/ChihabEddine98/AmazonAnalysis/blob/main/AmazonAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install webdriver-manager
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [5]:
# imports
import os
import requests as rq
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
import unicodedata
# end imports

# Part 01  : Data Gathering

In [6]:
# Settings 
reviews_div_cls = 'review'

prod_urls = {'informatique':[
    {
        'prod_url':
        'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/dp/B07X8NL7MW/ref=sr_1_7? __mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=Microsoft+Surface&qid=1616930888&  s=computers&sr=1-7',
    'rev_url': 'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/product-reviews/B07X8NL7MW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
    
    }
    ]
}

ds_cols = ['Rev_Title','Rev_Rate','Rev_Bdy','Rev_Hlp','Prod_Title','Prod_Brnd','Prod_Rate','Prod_Eval','Prod_Price','Category']


# End Settings

# Scrapping Product Information

In [7]:

# Scrapping Product Data 
def product_scrapper(driver,prod_category,prod_url):
    driver.get(prod_url)
    prod_soup = BeautifulSoup(driver.page_source,'html.parser')

    # Product Info
    prod_title = prod_soup.find('span',{'id' : 'productTitle' }).text
    prod_brand = prod_soup.find('a',{'id' : 'bylineInfo' }).text
    prod_rate = prod_soup.find('span',{'class' : 'a-icon-alt' }).text
    prod_eval = prod_soup.find('span',{'id' : 'acrCustomerReviewText' }).text
    prod_price = prod_soup.find('span',{'id' : 'priceblock_ourprice' }).text


    result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[prod_title,prod_brand,       prod_rate,prod_eval,prod_price,prod_category]))
    return result



## Test Product Scrapping

In [11]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])
        print(product)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


('Microsoft Surface Pro 7 Ordinateur Portable (Windows 10, écran tactile 12.3", Intel Core i5, 8Go RAM, 128Go SSD, Platine) PC Hybride polyvalent & performant', 'Marque : Microsoft Surface', '4,7 sur 5 étoiles', '3 274 évaluations', '991,98 €', 'informatique')


# Scrapping Reviews Data

In [12]:
# Scrapping Reviews Data
def reviews_scrapper(driver,prod,rev_url,ds):
    driver.get(rev_url)
    rev_soup = BeautifulSoup(driver.page_source,'html.parser')
    reviews_div = rev_soup.find_all('div',{'class':reviews_div_cls})

    for review in reviews_div:
        review_title = review.find('a',{'data-hook' : 'review-title'}).text
        review_rate = review.find('i',{'data-hook' : 'review-star-rating'}).text
        review_body = review.find('span',{'data-hook' : 'review-body'}).text
        review_help = review.find('span',{'data-hook' : 'helpful-vote-statement'}).text
        rev_result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[review_title,review_rate,review_body,review_help]))
        ds.append(rev_result+prod)

    return ds

## Test Reviews Scrapping

In [13]:
ds = []
for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])
        ds = reviews_scrapper(driver,product,prod['rev_url'],ds) 

df = pd.DataFrame(ds)
df.columns = ds_cols
df





,Rev_Title,Rev_Rate,Rev_Bdy,Rev_Hlp,Prod_Title,Prod_Brnd,Prod_Rate,Prod_Eval,Prod_Price,Category
0,Surface Pro 7ème génération,"4,0 sur 5 étoiles",Je possède la Surface Pro 6 (soit la généra...,155 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
1,Du reconditionné au prix du neuf!,"1,0 sur 5 étoiles",Produit bien reçu.Par contre on achète au pr...,110 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
2,THROTTING,"2,0 sur 5 étoiles",POUR LA VERSION I5 SEULEMENT : PHENOMENE DE TH...,72 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
3,ATTENTION AU TITRE TROMPEUR !,"2,0 sur 5 étoiles",Le titre fait référence à un Surface Pro 7 ...,98 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
4,C'est bien une version i7,"5,0 sur 5 étoiles",Contrairement au commentaire disant que c'est ...,44 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
5,Un excellent choix,"5,0 sur 5 étoiles","Pour faire court, sans aucun doute mon ""pc por...",23 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
6,Plus pertinent qu'une tablette sous Android.,"5,0 sur 5 étoiles",Belle bête ! J'essaie pour la 1ere fois Windo...,22 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
7,Eclan bleu systématique au démarrage ou dans...,"1,0 sur 5 étoiles","Sans qu'aucun logiciel n'ai été installé, j...",10 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
8,Incroyable,"4,0 sur 5 étoiles",Cette tablette est incroyable et presque parfa...,Une personne a trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
9,Tablette fonctionne bien,"1,0 sur 5 étoiles",UPDATE: La durée de vie de la batterie est de...,4 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
